In [1]:
# !pip install optimum -q
# !pip install --upgrade --quiet datasets[audio] transformers accelerate evaluate jiwer tensorboard
# !pip install pydub -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pathos 0.3.3 requires dill>=0.3.9, but you have dill 0.3.8 which is incompatible.
pathos 0.3.3 requires multiprocess>=0.70.17, but you have multiprocess 0.70.16 which is incompatible.


In [108]:
import json
import numpy as np
from IPython.display import Audio
import IPython.display as ipd
from transformers import BarkModel, AutoProcessor, AutoTokenizer
import torch
from tqdm import tqdm
import io
import pickle

In [109]:
device="cuda"

In [110]:
def generate_speaker1_audio(text):
    """Generate audio using Bark for Speaker 1"""
    inputs = bark_processor(text, voice_preset="v2/en_speaker_5").to(device)
    speech_output = bark_model.generate(**inputs, temperature=0.5, semantic_temperature=0.8)
    audio_arr = speech_output[0].cpu().numpy()
    return audio_arr, sampling_rate

In [111]:
def generate_speaker2_audio(text):
    """Generate audio using Bark for Speaker 2"""
    inputs = bark_processor(text, voice_preset="v2/en_speaker_6").to(device)
    speech_output = bark_model.generate(**inputs, temperature=0.5, semantic_temperature=0.8)
    audio_arr = speech_output[0].cpu().numpy()
    return audio_arr, sampling_rate


In [112]:
def numpy_to_audio_segment(audio_arr, sampling_rate):
    """Convert numpy array to AudioSegment"""
    # Convert to 16-bit PCM
    audio_int16 = (audio_arr * 32767).astype(np.int16)
    
    # Create WAV file in memory
    byte_io = io.BytesIO()
    wavfile.write(byte_io, sampling_rate, audio_int16)
    byte_io.seek(0)
    
    # Convert to AudioSegment
    return AudioSegment.from_wav(byte_io)

In [113]:
bark_processor = AutoProcessor.from_pretrained("suno/bark")
bark_model = BarkModel.from_pretrained("suno/bark", torch_dtype=torch.float16).to(device)

/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/transformers/models/encodec/modeling_encodec.py:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)


In [114]:
with open('./resources/podcast_ready_data.pkl', 'rb') as file:
    PODCAST_TEXT = pickle.load(file)

In [115]:
NEW_PODCAST_TEXT = [ast.literal_eval(a[:-2]) for a in PODCAST_TEXT.split('  ')[::2][1:]]
for text in NEW_PODCAST_TEXT:
    speaker, txt = text
    print(speaker)
    print(txt)
    

Speaker 1
Welcome to 'Mind Blown', the podcast where we explore the fascinating world of artificial intelligence and its impact on our daily lives. I'm your host, and I'm excited to have with me today, a brilliant mind who's new to the world of AI, but eager to learn. Welcome to the show!
Speaker 2
Thanks for having me! I'm super excited to be here. I have to admit, I'm a total newbie when it comes to AI. I mean, I've heard of it, but I don't really know what it's all about. So, let's dive in!
Speaker 1
Well, let's start with the basics. Artificial intelligence, or AI for short, refers to the development of computer systems that can perform tasks that typically require human intelligence, such as learning, problem-solving, and decision-making. Think of it like a super-smart robot that can learn from its mistakes and get better over time.
Speaker 2
Hmm, that sounds like science fiction. I mean, I've seen those movies where robots take over the world... (laughs)
Speaker 1
Well, we're not

In [116]:
from scipy.io import wavfile
from pydub import AudioSegment

In [117]:
sampling_rate = 24000
final_audio = None

# for speaker, text in tqdm(NEW_PODCAST_TEXT, desc="Generating podcast segments", unit="segment"):
#     print('----')
#     if speaker == "Speaker 1":
#         print(speaker)
#         print(text)
#         audio_arr, rate = generate_speaker1_audio(text)
#     else:  # Speaker 2
#         print(speaker)
#         print(text)
#         audio_arr, rate = generate_speaker2_audio(text)
#     print(audio_arr.shape)
        
#     # Convert to AudioSegment (pydub will handle sample rate conversion automatically)
#     audio_segment = numpy_to_audio_segment(audio_arr, rate)
    
#     # Add to final audio
#     if final_audio is None:
#         final_audio = audio_segment
#     else:
#         final_audio += audio_segment

In [118]:
audio_arr_list = []

for index in tqdm(range(len(NEW_PODCAST_TEXT))):
    speaker, text = NEW_PODCAST_TEXT[index]
    if speaker == "Speaker 1":
        print(speaker)
        print(text)
        audio_arr, rate = generate_speaker1_audio(text)
    else:  # Speaker 2
        print(speaker)
        print(text)
        audio_arr, rate = generate_speaker2_audio(text)
    print(audio_arr.shape)
    audio_arr_list.append(audio_arr)
    
concat_arr = np.concatenate(audio_arr_list, axis=0)
print(concat_arr.shape)

  0%|          | 0/25 [00:00<?, ?it/s]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Speaker 1
Welcome to 'Mind Blown', the podcast where we explore the fascinating world of artificial intelligence and its impact on our daily lives. I'm your host, and I'm excited to have with me today, a brilliant mind who's new to the world of AI, but eager to learn. Welcome to the show!


  4%|▍         | 1/25 [00:39<15:36, 39.00s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


(326080,)
Speaker 2
Thanks for having me! I'm super excited to be here. I have to admit, I'm a total newbie when it comes to AI. I mean, I've heard of it, but I don't really know what it's all about. So, let's dive in!


  8%|▊         | 2/25 [01:21<15:48, 41.25s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

(96000,)
Speaker 2
Bye!


 96%|█████████▌| 24/25 [12:29<00:16, 16.50s/it]The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


(61760,)
Speaker 1
Bye!


100%|██████████| 25/25 [12:35<00:00, 30.20s/it]

(48000,)
(6317120,)


In [ ]:
Audio(concat_arr, rate=sampling_rate)

In [ ]:
audio_data = Audio(concat_arr, rate=sampling_rate).data
with open("./resources/podcast.wav", "wb") as f:
    f.write(audio_data)

In [ ]:
# Audio(audio_arr, rate=sampling_rate)